In [12]:
import pandas as pd 
import numpy as np
import re

In [239]:
%pwd

'/Users/jieun/projects/M5_Idol_lyrics'

In [240]:
# !pip install soynlp
!pip show soynlp

Name: soynlp
Version: 0.0.46
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: /Users/jieun/anaconda3/lib/python3.6/site-packages
Requires: psutil, numpy
Required-by: 


In [271]:
data= pd.read_csv("song_data_fixed.csv")

In [272]:
song = pd.DataFrame(data)

In [273]:
song.head(3)

,Unnamed: 0,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
0,0,태지 보이스,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,NaN,269,서태지\r\r\n작사\r\r\n서태지\r\r\n작곡\r\r\n서태지\r\r\n편곡,갈수 없는 곳이란 우리에겐 없어\r\r\n보이는 길밖에도 세상은 있어\r\r\n언제...
1,1,시대유감(時代遺憾),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,타이틀 곡,"3,256",서태지\r\r\n작사\r\r\n서태지\r\r\n작곡\r\r\n서태지\r\r\n편곡,왜 기다려 왔잖아\r\r\n모든 삶을 포기하는 소리를\r\r\n이 세상이 모두 미쳐...
2,2,수시아(誰是我),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,NaN,488,서태지\r\r\n작사\r\r\n서태지\r\r\n작곡\r\r\n서태지\r\r\n편곡,이 세상 그 누구도 나를 닮을 순 없네\r\r\n날 세상에 알릴 거야\r\r\n나 ...


In [274]:
song.columns

Index(['Unnamed: 0', 'title', 'artist', 'album', 'release_date', 'song_genre',
       'is_title', 'like', 'creator', 'lyrics'],
      dtype='object')

In [275]:
song['artist'].value_counts()

동방신기           865
키스             839
슈퍼주니어          518
샤이니            469
쿨              375
신화             354
소녀시대           323
엑소             322
비원에이포          296
플라이 투 더 스카이    289
터보             285
구피             268
방탄소년단          241
빅뱅             232
에프티 아일랜드       230
투피엠            228
god            210
엔알지            199
룰라             198
인피니트           198
브라운 아이드 걸스     175
티아라            169
젝스키스           166
에이치오티          165
틴탑             149
핑클             144
에스이에스          142
버즈             142
쥬얼리            141
씨엔블루           140
              ... 
비투비-블루           5
타키온              5
소년24             4
키스 파이브           4
립버블              4
보너스베이비           4
브이엔티             4
에스아이에스           4
씨아이브이에이          3
구구단 세미나          3
인피니트F            3
언니쓰              3
엠앤엔              3
핫젝갓알지            3
바이칼              2
스피카.S            2
애플비              2
아이비아이            2
옆집소녀             2
구구단 오구오구         2
맙                2
레인보우 픽시     

In [276]:
song[song['artist'] == '키스'].head() # 키스는 누구인가? => 미국가수, 삭제예정

,Unnamed: 0,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
5088,5088,Rock And Roll All Nite,키스,Torcida 2018 - Nacional e Internacional,2018.06.01,World,`게임 쉬리` OST (`이젠` Metal Ver.) 와 동영상 수록,1,NaN,You show us everything you've got\r\r\nYou kee...
5089,5089,Rock And Roll All Nite,키스,New Year`s Eve Countdown,2017.12.15,기타,NaN,7,NaN,NaN
5090,5090,Calling Dr. Love,키스,Throwback Tunes: Classic Rock,2017.10.20,Pop,NaN,1,NaN,NaN
5091,5091,Rock And Roll All Nite,키스,Countdown 40th Anniversary,2016.11.24,Pop,타이틀 곡,11,NaN,NaN
5092,5092,Deuce,키스,"Radio Waves 1974-1988: The Very Best of Kiss, ...",2016.10.21,World,NaN,0,NaN,NaN


In [277]:
# 유니크한 가수의 수 
len(song['artist'].unique())

331

In [278]:
# 곡의 수 
song.shape

(19165, 10)

In [279]:
song['lyrics'].isnull().sum()

2788

In [281]:
song = song['lyrics'].dropna().reset_index(inplace=False) #inplace=True 를 하면 원래 데이터프레임에서 날라간다

In [283]:
song['lyrics'].isnull().sum()

0

## 텍스트 데이터 전처리

In [284]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [285]:
sample_content = song['lyrics'][100]
type(sample_content)

str

In [286]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'그대여 내게 아픈 말은 하지마 슬프지 않은 미소지며 잠시 떨려오는 마지막 눈빛으로 헤어짐을 간직하라는 이젠 네가없는 내가 힘들어 나를 포기할 수 있기에 너를 내 꿈속에 스치듯 찾아들면 그때 갈께 너의 곁으로 떠난 그대여 슬퍼 하지마 우리 잃어버린 사랑은 영원할꺼야 다른 세상에 기대고 있어 아직 살아있는 나를 기다려줘 우리 헤어짐은 이대로 간직할께 눈물에 흘리지 않고 그리고 바라보면 언제나 내가 있어 그대 떠난 그대여 슬퍼 하지마 우리 잃어버린 사랑은 영원할꺼야 다른 세상에 기대고있어 아직 살아있는 나를 기다려줘 우리 헤어짐은 이대로 간직할게 눈물에 흘리지 않고 그리워 바라보면 언제나 내가있어 그대'

In [287]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = song['lyrics'].apply(preprocessing)

CPU times: user 2.58 s, sys: 33.2 ms, total: 2.61 s
Wall time: 2.62 s


## 토크나이징

In [288]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [289]:
# 전처리 이후의 샘플 텍스트로 토큰화
tokened_content = tokenizer.tokenize(sample_content)
tokened_content

['그대여',
 '내게',
 '아픈',
 '말은',
 '하지마',
 '슬프지',
 '않은',
 '미소지며',
 '잠시',
 '떨려오는',
 '마지막',
 '눈빛으로',
 '헤어짐을',
 '간직하라는',
 '이젠',
 '네가없는',
 '내가',
 '힘들어',
 '나를',
 '포기할',
 '수',
 '있기에',
 '너를',
 '내',
 '꿈속에',
 '스치듯',
 '찾아들면',
 '그때',
 '갈께',
 '너의',
 '곁으로',
 '떠난',
 '그대여',
 '슬퍼',
 '하지마',
 '우리',
 '잃어버린',
 '사랑은',
 '영원할꺼야',
 '다른',
 '세상에',
 '기대고',
 '있어',
 '아직',
 '살아있는',
 '나를',
 '기다려줘',
 '우리',
 '헤어짐은',
 '이대로',
 '간직할께',
 '눈물에',
 '흘리지',
 '않고',
 '그리고',
 '바라보면',
 '언제나',
 '내가',
 '있어',
 '그대',
 '떠난',
 '그대여',
 '슬퍼',
 '하지마',
 '우리',
 '잃어버린',
 '사랑은',
 '영원할꺼야',
 '다른',
 '세상에',
 '기대고있어',
 '아직',
 '살아있는',
 '나를',
 '기다려줘',
 '우리',
 '헤어짐은',
 '이대로',
 '간직할게',
 '눈물에',
 '흘리지',
 '않고',
 '그리워',
 '바라보면',
 '언제나',
 '내가있어',
 '그대']

In [290]:
%time tokens = sentences.apply(tokenizer.tokenize)
tokens[:3]

CPU times: user 23.8 s, sys: 292 ms, total: 24.1 s
Wall time: 24.3 s


0    [갈수, 없는, 곳이란, 우리에겐, 없어, 보이는, 길밖에도, 세상은, 있어, 언제...
1    [왜, 기다려, 왔잖아, 모든, 삶을, 포기하는, 소리를, 이, 세상이, 모두, 미...
2    [이, 세상, 그, 누구도, 나를, 닮을, 순, 없네, 날, 세상에, 알릴, 거야,...
Name: lyrics, dtype: object

## Word2vec 모델 만들기

In [129]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.

import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [130]:
# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(tokens, min_count=1)  # word2vec에 tokens을 넣어주고, min_count =1 로 해주면 최소한 1번들어가 있으면 뽑히게 한다. 

model

2018-09-11 11:44:14,235 : INFO : 'pattern' package not found; tag filters are not available for English
2018-09-11 11:44:14,243 : INFO : collecting all words and their counts
2018-09-11 11:44:14,245 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-11 11:44:14,724 : INFO : PROGRESS: at sentence #10000, processed 2185503 words, keeping 118973 word types
2018-09-11 11:44:15,096 : INFO : collected 162387 word types from a corpus of 3861201 raw words and 16377 sentences
2018-09-11 11:44:15,099 : INFO : Loading a fresh vocabulary
2018-09-11 11:44:15,634 : INFO : min_count=1 retains 162387 unique words (100% of original 162387, drops 0)
2018-09-11 11:44:15,635 : INFO : min_count=1 leaves 3861201 word corpus (100% of original 3861201, drops 0)
2018-09-11 11:44:16,140 : INFO : deleting the raw counts dictionary of 162387 items
2018-09-11 11:44:16,144 : INFO : sample=0.001 downsamples 35 most-common words
2018-09-11 11:44:16,146 : INFO : downsampling leaves esti

In [131]:
# Word2vec 모델 이름을 지정하고 저장한다.
model_name = 'word2vec_first'  # 로컬이면 내 PC에 저장하므로 계속 학습시키지 않고 그냥 바로 불러와서 가져다가 쓸 수 있다. 
model.save(model_name)

2018-09-11 11:45:09,714 : INFO : saving Word2Vec object under word2vec_first, separately None
2018-09-11 11:45:09,716 : INFO : storing np array 'vectors' to word2vec_first.wv.vectors.npy
2018-09-11 11:45:09,795 : INFO : not storing attribute vectors_norm
2018-09-11 11:45:09,798 : INFO : storing np array 'syn1neg' to word2vec_first.trainables.syn1neg.npy
2018-09-11 11:45:09,905 : INFO : not storing attribute cum_table
2018-09-11 11:45:10,362 : INFO : saved word2vec_first


In [132]:
# 워드투벡한 단어의 사전 수 보기 
len(model.wv.vocab)

162387

In [161]:
# 단어 사전에서 상위 10개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:10]

['I', 'you', '내', '난', 'me', 't', '널', '날', '수', 'it']

In [160]:
# 가장 유사한 단어를 추출
model.wv.most_similar('너')

[('완성되지', 0.5744380354881287),
 ('억하심정인지', 0.5725435614585876),
 ('너야', 0.5635101199150085),
 ('물어볼게', 0.5609223246574402),
 ('있기', 0.5590577125549316),
 ('털고도', 0.5531821846961975),
 ('피땀에', 0.551423966884613),
 ('옛정', 0.5352168083190918),
 ('네가', 0.5340349674224854),
 ('하나만', 0.5245047807693481)]

In [135]:
model.wv.most_similar('사랑')

[('사랑은', 0.6728729009628296),
 ('사랑을', 0.6576943397521973),
 ('사랑으로', 0.6437615156173706),
 ('행복을', 0.633502721786499),
 ('사랑아', 0.6203153729438782),
 ('마음', 0.613693356513977),
 ('바래', 0.6091277599334717),
 ('바램', 0.6004109382629395),
 ('없는거라고', 0.5848444700241089),
 ('평생', 0.5808133482933044)]

In [136]:
model.wv.most_similar('여자')

[('남자', 0.84284508228302),
 ('남자를', 0.7608194351196289),
 ('놈이', 0.7475835084915161),
 ('여자는', 0.7387921214103699),
 ('여자야', 0.7235146760940552),
 ('남자가', 0.7207379937171936),
 ('놈', 0.7175110578536987),
 ('여잘', 0.7169695496559143),
 ('남자야', 0.7162264585494995),
 ('여자가', 0.7151209115982056)]

In [137]:
model.wv.most_similar('남자')

[('여자', 0.84284508228302),
 ('여잔', 0.7740013003349304),
 ('남자가', 0.7644602060317993),
 ('여자야', 0.7559769153594971),
 ('놈', 0.7550831437110901),
 ('사람은', 0.7539772391319275),
 ('여잘', 0.753886342048645),
 ('여자를', 0.7496883273124695),
 ('놈이', 0.742708683013916),
 ('남자를', 0.7252805233001709)]

In [138]:
model.wv.most_similar('baby')

[('lady', 0.6719524264335632),
 ('honey', 0.6599240899085999),
 ('babe', 0.6590299606323242),
 ('뿐이기에', 0.6517203450202942),
 ('shiMega', 0.6356972455978394),
 ('girl', 0.6356855630874634),
 ('Baby', 0.6232913136482239),
 ('shiranpuri', 0.6048318147659302),
 ('눈뜨지마', 0.5972138047218323),
 ('아루요니', 0.596859335899353)]

In [141]:
model.wv.most_similar('JYP')

[('멋져다리는', 0.8559885025024414),
 ('만점에', 0.8548582792282104),
 ('점그녀의', 0.8440045714378357),
 ('다리는', 0.8331058621406555),
 ('머릿결날리는', 0.8326587677001953),
 ('유루사레루나라', 0.8284714818000793),
 ('터프한', 0.8232554197311401),
 ('요리', 0.822808563709259),
 ('요루와후카맛테쿠요', 0.8211950063705444),
 ('쭉빠진', 0.8158367276191711)]

In [142]:
model.wv.most_similar('이별')

[('이별은', 0.7862210869789124),
 ('않잖냐', 0.758197546005249),
 ('슬픈', 0.7512339353561401),
 ('끝내', 0.7437535524368286),
 ('얘기', 0.7389219999313354),
 ('이별을', 0.7288054823875427),
 ('이별이', 0.7193502187728882),
 ('아니였겠지', 0.7178677916526794),
 ('어리석은', 0.7177433371543884),
 ('지나간', 0.712112307548523)]

In [144]:
model.wv.most_similar('sexy')

[('pretty', 0.7721747159957886),
 ('bling', 0.683236837387085),
 ('gorgeous', 0.6781123280525208),
 ('Sexy', 0.6679773926734924),
 ('ome', 0.6646952033042908),
 ('sugar', 0.6584035158157349),
 ('caress', 0.6559441685676575),
 ('Warring', 0.6523290872573853),
 ('ter', 0.6519011855125427),
 ('Paras', 0.6474953889846802)]

In [146]:
model.wv.most_similar('기다려')

[('기다릴게', 0.7881777882575989),
 ('기다렸어', 0.7421151399612427),
 ('보내고선', 0.6889650821685791),
 ('사랑할게', 0.687706470489502),
 ('약속해', 0.6874222159385681),
 ('위해서', 0.6810830235481262),
 ('기다려왔어', 0.6747235059738159),
 ('기도해', 0.6743015646934509),
 ('변함없이', 0.6708771586418152),
 ('원해', 0.6632115244865417)]

In [147]:
model.wv.most_similar('행복')

[('시간만큼', 0.786211371421814),
 ('그동안', 0.785946249961853),
 ('오래도록', 0.7828967571258545),
 ('친구가', 0.770990788936615),
 ('없는거라고', 0.7703820466995239),
 ('이별을', 0.7634376287460327),
 ('오래전', 0.7602222561836243),
 ('힘겨운', 0.757683277130127),
 ('만들어지고', 0.7558469772338867),
 ('사랑과', 0.7513598799705505)]

In [148]:
model.wv.most_similar('여자친구')

[('예쁘더라', 0.8774560689926147),
 ('홀렸네', 0.8485866189002991),
 ('연락했니', 0.8478046655654907),
 ('얘기만', 0.8463076949119568),
 ('수첩이야', 0.8446948528289795),
 ('번호', 0.8440446853637695),
 ('연락처에', 0.8438520431518555),
 ('그녀완', 0.8435104489326477),
 ('소식', 0.8411042094230652),
 ('멀리한건', 0.8399530649185181)]

In [149]:
model.wv.most_similar('남자친구')

[('행세', 0.8161464333534241),
 ('수이수', 0.8149913549423218),
 ('여자친구', 0.8035971522331238),
 ('소식', 0.7937813997268677),
 ('끊어버리고', 0.7933464050292969),
 ('Isis', 0.7933018207550049),
 ('쿠라야미니이루요데', 0.7877689599990845),
 ('번호', 0.7852747440338135),
 ('올라온', 0.7846376895904541),
 ('피곤해', 0.7839388847351074)]

In [150]:
model.wv.most_similar('친구')

[('얘기', 0.7857072949409485),
 ('좋아요로', 0.747502326965332),
 ('친구가', 0.7441698312759399),
 ('친구들', 0.7215195894241333),
 ('얘길', 0.7154452204704285),
 ('빅뱅이', 0.7145744562149048),
 ('얘기를', 0.7116910219192505),
 ('애기가', 0.6933189630508423),
 ('남자들', 0.691458523273468),
 ('수는없어', 0.6835935711860657)]

In [159]:
model.wv.most_similar('최고')

[('정신력도', 0.8311450481414795),
 ('뒤태도', 0.7932344675064087),
 ('이야', 0.7747499346733093),
 ('스케일도', 0.773597240447998),
 ('다른데', 0.7732925415039062),
 ('규모도', 0.759735107421875),
 ('난리나', 0.752419114112854),
 ('브이아이피', 0.7524104714393616),
 ('누가봐도', 0.7455583810806274),
 ('오왜', 0.7371297478675842)]